In [1]:
# FINAL PROJECT DATA ENGINEERING (Long Trade Program Script)
# Author - Priya Kundu and Nikitaa Kenkre
# Date of Program Run - 11th May 2023 (3pm to 7:30am)
# Date of Presentation - 12th May 2023 (11am)
# Semester Spring 2023
# By Professor Carlos De Oliveira

# Importing Packages and Setting Parameters

In [2]:
import oandapyV20
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import pandas as pd
import datetime as dt
import time
from datetime import datetime

In [3]:
# OANDA account details
access_token = "681073222fb4cf28caa21c5d2bec7322-4a90092d303c32495bdc15a63a9708ee"
accountID = "101-001-25499063-005"
#api = oandapyV20.API(access_token=access_token, environment="practice")
try:
    client = oandapyV20.API(access_token=access_token, environment="practice")
except V20Error as e:
    print("Error: {}".format(e))

In [4]:
# specify the instrument to trade
instrument = "EUR_GBP"

In [5]:
# set the order parameters
order_quantity = 1000 # start with 1000 units
total_order_quantity = 100000 # the total order quantity
current_executed_quantity = 0 # keep track of how many units have been executed so far
non_executed_quantity = 0 # keep track of the units that were not executed in the previous window
average_execution_price = 0 # initialize the average execution price to 0
total_average_execution_price = 0 # initialize the total average execution price to 0

In [6]:
# setting the required time frames
first_execution_start_time = dt.datetime.now().replace(hour=15, minute=0, second=0, microsecond=0)
first_execution_end_time = dt.datetime.now().replace(hour=17, minute=0, second=0, microsecond=0)

second_execution_start_time = dt.datetime.now().replace(hour=19, minute=0, second=0, microsecond=0)
second_execution_end_time = dt.datetime.now().replace(hour=22, minute=0, second=0, microsecond=0)

third_execution_start_time = dt.datetime.now().replace(hour=23, minute=0, second=0, microsecond=0)
third_execution_end_time = dt.datetime.now().replace(hour=1, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

fourth_execution_start_time = dt.datetime.now().replace(hour=3, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the start time
fourth_execution_end_time = dt.datetime.now().replace(hour=6, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

In [7]:
# define a function to execute orders
def execute_order(units, instrument, side, trades):
    # create a market order request
    direction = 1 if side == "BUY" else -1
    mo = MarketOrderRequest(
        instrument=instrument,
        units=str(int(units)*direction),
        takeProfitOnFill=None,
        stopLossOnFill=None,
    )
    # send the order request
    r = orders.OrderCreate(accountID, data=mo.data)
    response = client.request(r)
    if "orderFillTransaction" in response:
        trades.append({
            "Timestamp": dt.datetime.now(), 
            "Order_ID": float(response["orderFillTransaction"]["orderID"]),
            "Instrument": instrument,
            "Price": float(response["orderFillTransaction"]["price"]),
            "Units": int(units),
            "Side": side
        })
    else:
        # Handle the case where the order is not filled
        if "orderCancelTransaction" in response:
            print("Order canceled due to:", response["orderCancelTransaction"]["reason"])
        else:
            print("No cancel transaction found.")

In [8]:
# make a function to get price that can be used to compare with the average price before execution takes place
def get_rate():
    price_request = pricing.PricingInfo(accountID=accountID, params={'instruments': instrument})
    response = client.request(price_request)
    return float(response['prices'][0]['asks'][0]['price'])

# First Window

In [9]:
# execute the first window
while dt.datetime.now() < first_execution_start_time:
    time.sleep(1)

batch1_trades = []  # create a list to store the prices

while dt.datetime.now() >= first_execution_start_time and dt.datetime.now() <= first_execution_end_time:
    for i in range(20): # 20 iterations to execute 1000 units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.2:       
            response = execute_order(order_quantity, instrument, "BUY", batch1_trades)
            current_executed_quantity += order_quantity
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval.") 

Successful execution for 0 th interval.
Successful execution for 1 th interval.
Successful execution for 2 th interval.
Successful execution for 3 th interval.
Successful execution for 4 th interval.
Successful execution for 5 th interval.
Successful execution for 6 th interval.
Successful execution for 7 th interval.
Successful execution for 8 th interval.
Successful execution for 9 th interval.
Successful execution for 10 th interval.
Successful execution for 11 th interval.
Successful execution for 12 th interval.
Successful execution for 13 th interval.
Successful execution for 14 th interval.
Successful execution for 15 th interval.
Successful execution for 16 th interval.
Successful execution for 17 th interval.
Successful execution for 18 th interval.
Successful execution for 19 th interval.


In [10]:
# convert the list of prices into a DataFrame
df1 = pd.DataFrame(batch1_trades)

df1

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 15:00:01.129669,18.0,EUR_GBP,0.87272,1000,BUY
1,2023-05-11 15:06:01.229916,26.0,EUR_GBP,0.87273,1000,BUY
2,2023-05-11 15:12:01.343148,32.0,EUR_GBP,0.87279,1000,BUY
3,2023-05-11 15:18:01.423751,36.0,EUR_GBP,0.87272,1000,BUY
4,2023-05-11 15:24:01.547182,44.0,EUR_GBP,0.87261,1000,BUY
5,2023-05-11 15:30:01.656702,50.0,EUR_GBP,0.87253,1000,BUY
6,2023-05-11 15:36:01.752260,56.0,EUR_GBP,0.87262,1000,BUY
7,2023-05-11 15:42:01.832240,62.0,EUR_GBP,0.87260,1000,BUY
8,2023-05-11 15:48:01.920696,68.0,EUR_GBP,0.87266,1000,BUY
9,2023-05-11 15:54:02.030762,72.0,EUR_GBP,0.87267,1000,BUY


In [11]:
average_execution_price1 = df1['Price'].sum() / len(df1)

average_execution_price1

0.8725594999999998

In [12]:
current_executed_quantity

20000

# Second Window

In [13]:
# execute the second window
while dt.datetime.now() < second_execution_start_time:
    time.sleep(1)

batch2_trades = []  # create a list to store the prices

while dt.datetime.now() >= second_execution_start_time and dt.datetime.now() <= second_execution_end_time:
    for i in range(30): # 30 iterations to execute 1000 units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.5:
            if get_rate() >= average_execution_price1:
                execute_order(order_quantity, instrument, "BUY", batch2_trades)
                current_executed_quantity += order_quantity
                print("Trade took place!")
            else:
                print("Trade did not take place as Price > Average Price.")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval.")

Trade did not take place as Price > Average Price.
Successful execution for 0 th interval.
Trade took place!
Successful execution for 1 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 2 th interval.
Trade took place!
Successful execution for 3 th interval.
Trade took place!
Successful execution for 4 th interval.
Trade took place!
Successful execution for 5 th interval.
Trade took place!
Successful execution for 6 th interval.
Trade took place!
Successful execution for 7 th interval.
Trade took place!
Successful execution for 8 th interval.
Trade took place!
Successful execution for 9 th interval.
Trade took place!
Successful execution for 10 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 11 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 12 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 13 th interval.
Trade did not take 

In [14]:
# convert the list of prices into a DataFrame
df2 = pd.DataFrame(batch2_trades)

df2

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 19:06:00.415881,139.0,EUR_GBP,0.87256,1000,BUY
1,2023-05-11 19:18:00.692602,142.0,EUR_GBP,0.87260,1000,BUY
2,2023-05-11 19:24:00.843537,145.0,EUR_GBP,0.87262,1000,BUY
3,2023-05-11 19:30:01.025711,148.0,EUR_GBP,0.87262,1000,BUY
4,2023-05-11 19:36:01.204333,151.0,EUR_GBP,0.87260,1000,BUY
5,2023-05-11 19:42:01.395665,154.0,EUR_GBP,0.87257,1000,BUY
6,2023-05-11 19:48:01.572463,157.0,EUR_GBP,0.87259,1000,BUY
7,2023-05-11 19:54:01.709192,160.0,EUR_GBP,0.87260,1000,BUY
8,2023-05-11 20:00:01.930698,163.0,EUR_GBP,0.87257,1000,BUY
9,2023-05-11 21:06:03.124271,187.0,EUR_GBP,0.87259,1000,BUY


In [15]:
# concatenate the two dataframes vertically
new_df = pd.concat([df1, df2], axis=0)

In [16]:
# calculate the total average execution price
average_execution_price2 = new_df['Price'].sum() / len(new_df)

average_execution_price2

0.8725902631578948

In [17]:
current_executed_quantity

38000

In [18]:
non_executed_quantity2 = total_order_quantity * 0.5 - current_executed_quantity

non_executed_quantity2

12000.0

# Third Window

In [20]:
# execute the third window
while dt.datetime.now() < third_execution_start_time:
    time.sleep(1)

batch3_trades = []  # create a list to store the prices

while dt.datetime.now() >= third_execution_start_time and dt.datetime.now() <= third_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity2 + (total_order_quantity * 0.2)) / 20)
    for i in range(20): # 20 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.7:
            if get_rate() >= average_execution_price2:
                response = execute_order(order_quantity, instrument, "BUY", batch3_trades)
                current_executed_quantity += order_quantity
                print("Trade took place!")
            else:
                print("Trade did not take place as Price > Average Price.")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval.")

Trade took place!
Successful execution for 0 th interval.
Trade took place!
Successful execution for 1 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 2 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 3 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 4 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 5 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 6 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 7 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 8 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 9 th interval.
Trade did not take place as Price > Average Price.
Successful execution for 10 th interval.
Trade did not take place as Price > Average Price.
Successful ex

In [21]:
# convert the list of prices into a DataFrame
df3 = pd.DataFrame(batch3_trades)

df3

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 23:00:00.782822,241.0,EUR_GBP,0.87279,1600,BUY
1,2023-05-11 23:06:00.990212,247.0,EUR_GBP,0.87271,1600,BUY


In [22]:
# concatenate the two dataframes vertically
updated_df = pd.concat([new_df, df3], axis=0)

In [23]:
# calculate the total average execution price
average_execution_price3 = updated_df['Price'].sum() / len(updated_df)

average_execution_price3

0.87259825

In [24]:
current_executed_quantity

41200

In [25]:
non_executed_quantity3 = total_order_quantity * 0.7 - current_executed_quantity

non_executed_quantity3

28800.0

# Fourth Window

In [26]:
# execute the fourth window
while dt.datetime.now() < fourth_execution_start_time:
    time.sleep(1)

batch4_trades = []  # create a list to store the prices

while dt.datetime.now() >= fourth_execution_start_time and dt.datetime.now() <= fourth_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity3 + (total_order_quantity * 0.3)) / 30)
    for i in range(30): # 30 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity:
            if get_rate() >= average_execution_price3:
                response = execute_order(order_quantity, instrument, "BUY", batch4_trades)
                current_executed_quantity += order_quantity
                print("Trade took place!")
            else:
                print("Trade did not take place as Price < Average Price.")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval.")

Trade did not take place as Price < Average Price.
Successful execution for 0 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 1 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 2 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 3 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 4 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 5 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 6 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 7 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 8 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 9 th interval.
Trade did not take place as Price < Average Price.
Successful execution for 10 th interval

In [27]:
# convert the list of prices into a DataFrame
df4 = pd.DataFrame(batch4_trades)

df4

""


In [28]:
# concatenate the two dataframes vertically
final_df = pd.concat([updated_df, df4], axis=0)

In [29]:
# calculate the total average execution price
final_average_execution_price = updated_df['Price'].sum() / len(updated_df)

final_average_execution_price

0.87259825

In [30]:
current_executed_quantity

41200

In [31]:
non_executed_quantity_final = total_order_quantity - current_executed_quantity

non_executed_quantity_final

58800

In [32]:
final_df.to_csv("Long_Final_Executed_Data.csv", index=False)

# Final Extensions

In [33]:
final_df

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 15:00:01.129669,18.0,EUR_GBP,0.87272,1000,BUY
1,2023-05-11 15:06:01.229916,26.0,EUR_GBP,0.87273,1000,BUY
2,2023-05-11 15:12:01.343148,32.0,EUR_GBP,0.87279,1000,BUY
3,2023-05-11 15:18:01.423751,36.0,EUR_GBP,0.87272,1000,BUY
4,2023-05-11 15:24:01.547182,44.0,EUR_GBP,0.87261,1000,BUY
5,2023-05-11 15:30:01.656702,50.0,EUR_GBP,0.87253,1000,BUY
6,2023-05-11 15:36:01.752260,56.0,EUR_GBP,0.87262,1000,BUY
7,2023-05-11 15:42:01.832240,62.0,EUR_GBP,0.87260,1000,BUY
8,2023-05-11 15:48:01.920696,68.0,EUR_GBP,0.87266,1000,BUY
9,2023-05-11 15:54:02.030762,72.0,EUR_GBP,0.87267,1000,BUY


In [34]:
current_executed_quantity

41200

In [35]:
non_executed_quantity_final

58800

In [36]:
if non_executed_quantity_final > 0:
    
    # Wait for 30 minutes after last execution window
    extension1_end_time = fourth_execution_end_time + dt.timedelta(minutes=30)
    while dt.datetime.now() < extension1_end_time:
        time.sleep(1)

    extension1_executed_trades = []
    new_order_quantity = int(non_executed_quantity_final * 0.5)

    # Check condition and execute trade
    if get_rate() >= final_average_execution_price:
        execute_order(new_order_quantity, instrument, "BUY", extension1_executed_trades)
        print("Traded 50% of the Non Executed Quantity in one go as Price >= Average Price")
        
    else:
        print("Trading did not take place as Price < Average Price.")
        
else:
    print("Trading did not take place as the entire quantity was executed in the previous four execution windows.")

Trading did not take place as Price < Average Price.


In [37]:
# convert the list of prices into a DataFrame
extension1_df = pd.DataFrame(extension1_executed_trades)

extension1_df

""


In [38]:
if non_executed_quantity_final > 0:
    
    # Wait for 60 minutes after last execution window
    extension2_end_time = fourth_execution_end_time + dt.timedelta(minutes=60)
    while dt.datetime.now() < extension2_end_time:
        time.sleep(1)

    extension2_executed_trades = []

    # Check condition and execute trade
    if not extension1_df.empty:
        # If current Price is more than the Average Execution Price
        if get_rate() >= final_average_execution_price:
            # Buy remaining 50% of non-executed amount
            execute_order(new_order_quantity, instrument, "BUY", extension2_executed_trades)
            print("Traded remaining 50% of the Non Executed Quantity in one go as Price >= Average Price and Extention1 was executed.")
        # If If current Price is less than the Average Execution Price
        else:
            # Buy remaining 50% of non-executed amount
            execute_order(new_order_quantity, instrument, "BUY", extension2_executed_trades)
            print("Traded remaining 50% of the Non Executed Quantity in one go as Price < Average Price and Extention1 was executed.")

    else:
        # Check if current price is below average price, then start selling
        if get_rate() < final_average_execution_price:
            # Sell total executed units every one-minute (TWAP) whatever the price is until 20% of initial order is reached
            required_quantity = current_executed_quantity - (total_order_quantity * 0.2)
            executed_quantity = 0

            # Keep selling until executed_quantity < new_order_size
            while executed_quantity < required_quantity:
                execute_order(order_quantity, instrument, "SELL", extension2_executed_trades)
                executed_quantity += order_quantity
                time.sleep(60)
                print("Successful execution of trade!")
            print("Order quantity reached below 20% of the initial order.")
            
        else:
            print ("Trading did not take place as Price > Average Price.")
            
else:
    print("Trading did not take place as the entire quantity was executed in the previous four execution windows.")

Successful execution of trade!
Successful execution of trade!
Successful execution of trade!
Successful execution of trade!
Successful execution of trade!
Successful execution of trade!
Successful execution of trade!
Successful execution of trade!
Successful execution of trade!
Successful execution of trade!
Successful execution of trade!
Order quantity reached below 20% of the initial order.


In [39]:
# convert the list of prices into a DataFrame
extension2_df = pd.DataFrame(extension2_executed_trades)

extension2_df

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-12 07:00:00.923823,391.0,EUR_GBP,0.87000,1960,SELL
1,2023-05-12 07:01:01.035963,396.0,EUR_GBP,0.86998,1960,SELL
2,2023-05-12 07:02:01.144417,398.0,EUR_GBP,0.86981,1960,SELL
3,2023-05-12 07:03:01.267009,400.0,EUR_GBP,0.86978,1960,SELL
4,2023-05-12 07:04:01.409559,402.0,EUR_GBP,0.86980,1960,SELL
5,2023-05-12 07:05:01.528046,404.0,EUR_GBP,0.86981,1960,SELL
6,2023-05-12 07:06:01.677293,406.0,EUR_GBP,0.86988,1960,SELL
7,2023-05-12 07:07:01.825071,408.0,EUR_GBP,0.86986,1960,SELL
8,2023-05-12 07:08:01.999038,410.0,EUR_GBP,0.86984,1960,SELL
9,2023-05-12 07:09:02.101608,412.0,EUR_GBP,0.86981,1960,SELL


# Creating and Updating the Database

In [40]:
from pymongo import MongoClient

# MongoDb Setup
client = MongoClient('mongodb://localhost:27017/')
db = client["TWAP_Forex_Trading"]

In [41]:
collection_name = "Long_Trade_EURGBP"
collection = db[collection_name]
collection.insert_many(final_df.to_dict("records") + extension1_df.to_dict("records") + extension2_df.to_dict("records"))

# Creating csv File

In [44]:
# Create a dictionary with your data
data = {'Interval': ['Interval 1', 'Interval 2', 'Interval 3', 'Interval 4'],
        'Executed Quantity': [20000, 18000, 3200, 0], 
        'Percentage of Executed Quantity': [100, 60, 16, 0],
        'Non Executed Quantity': [0, 12000, 16800, 30000],
        'Percentage of Non Executed Quantity': [0, 40, 84, 100]}

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data)

# Add some sample data for the last three columns
df['Average Execution Price'] = [average_execution_price1, average_execution_price2, average_execution_price3, final_average_execution_price]

# Convert the DataFrame to a CSV file
df.to_csv('Final Project - Long - Execution Output Vectors.csv', index=False)